In [1]:
import re
import pandas as pd
import numpy as np

# Input(s)

In [2]:
#My input, the rest are for testing purposes
inp = """Q: q0, q1, q2, q3, q4,
         E: a, b, c,
         F: q4,
         D: {
         q0, a, q1,
         q1, b, q2,
         q2, c, q0,
         q1, b, q3,
         q3, a, q4,
         q3, b, q0
         }
         """

In [229]:
inp = """
        Q: q0, q1, q2,
        E: a, b, c,
        F: q2,
        D: {
        q0, a, q0,
        q0, b, q1,
        q1, c, q1,
        q1, c, q2,
        q2, a, q0,
        q1, a, q1
        }
        """

In [261]:
inp = """
        Q: q0, q1, q2, q3, q4,
        E: a, b, c,
        F: q4, 
        D: {
        q0, a, q1,
        q1, b, q2, 
        q1, b, q3,
        q2, c, q3,
        q3, a, q3,
        q3, b, q4
        }
        """

# FininteAutomaton Class

In [36]:
class FiniteAutomaton():
    def __init__(self):
        self.Q = []
        self.E = []
        self.F = []
        self.D = []
        self.nfa = {}
        self.dfa = {}
        self.nfa_table = []
        self.dfa_table = []
    
    def read_input(self, inp):
#       some clever regex stuff to split the automata into a list of lists of symbols  
        lis = re.split(':|\n', inp)[: -1]
        automata = []
        for item in lis:
            if (len([symbol for symbol in re.split('\W+', item) if symbol != '']) != 0):
                automata += [[symbol for symbol in re.split('\W+', item) if symbol != '']]
        print(automata)
        
        for i in range(len(automata)):
            if automata[i][0] == "Q":
                self.Q = automata[i + 1]
            if automata[i][0] == "E":
                self.E = automata[i + 1]
            if automata[i][0] == "F":
                self.F = automata[i + 1]
            if automata[i][0] == "D":
                self.D = [automata[j] for j in range(i + 1, i + 1 + len(automata) - 7)]
                
        for item in self.D:
            flag = 1
            if item[0] not in self.nfa.keys():
                self.nfa[item[0]] = {item[1]: [item[2]]}
            else:
                for key in self.nfa[item[0]].keys():
                    if key == item[1]:
                        self.nfa[item[0]][key] += [item[2]]
                        flag = 0
                if flag == 1:
                    self.nfa[item[0]][item[1]] = [item[2]]
                    
        self.create_nfa_table()
                    
                
    
    def create_nfa_table(self):
        tabel = np.empty((len(self.Q), len(self.E)))
        tabel[:] = np.nan
        tabel = pd.DataFrame(tabel, index=self.Q, columns=self.E)

        for key in self.nfa.keys():
            for inner_key in self.nfa[key].keys():
                tabel.loc[key, inner_key] = "".join([str(item) for item in self.nfa[key][inner_key]])

        tabel = tabel.fillna("")
            
        self.nfa_table = tabel
#         print(self.nfa_table)
        
    def print_dfa(self):
        print(self.dfa_table)
        
    def print_nfa():
        print(self.nfa_table)
        
    def to_dfa(self):
        self.dfa_table = pd.DataFrame(np.empty((1, len(self.E))), index=["q0"], columns=self.E)
        self.dfa_table.loc["q0", :] = self.nfa_table.loc["q0", :]
        to_visit = ["q0"]
        self.dfa["q0"] = self.nfa["q0"]
        nfa_table_copy = self.nfa_table.copy()
        
        while True:
            #exit condition out of while loop
            if len(to_visit) == 0:
                break
                
            for q in to_visit:
                if q not in nfa_table_copy.index:
                    sub_qs = re.findall("q[0-9]+", q)
                    nfa_table_copy.loc[q, :] = nfa_table_copy.loc[sub_qs, :].sum()

                to_visit = []
                for key in nfa_table_copy.columns:
                    if nfa_table_copy.loc[q, key] != "":
                        node_list = re.findall("q[0-9]+", nfa_table_copy.loc[q, key])
                        node_list_concat = nfa_table_copy.loc[q, key]
                        node_list_set = set(node_list)
                        #using sets to make sure qxqy is treated the same as qyqx etc.
                        list_index = [set(re.findall("q[0-9]+", index)) for index in self.dfa_table.index]

                        if node_list_set not in list_index:
                            self.dfa_table.loc[node_list_concat, :] = self.nfa_table.loc[node_list, :].sum()
                            to_visit += [node_list_concat]
                            
            self.dfa_table_to_dict()
            
    def dfa_table_to_dict(self):            
        self.dfa = {}
        for index in self.dfa_table.index:
            for column in self.dfa_table.columns:
                if self.dfa_table.loc[index, column] != "":
                    if index not in self.dfa.keys():
                        self.dfa[index] = {column : re.findall("q[0-9]+", self.dfa_table.loc[index, column])}
                    else:
                        if column not in self.dfa[index].keys():
                            self.dfa[index][column] = re.findall("q[0-9]+", self.dfa_table.loc[index, column])
                        else:
                            self.dfa[index][column] += [re.findall("q[0-9]+", self.dfa_table.loc[index, column])]
                            
    def check_string(self, string):
        try:
            start = "q0"
            for character in string:
                if character in self.dfa[start].keys():
                    start = "".join(self.dfa[start][character])
                else:
                    return False

            #checking if last accessed node was a terminal
            for q in re.findall("q[0-9]+", start):
                if q in self.F:
                    return True
                else:
                    return False
        except:
            return False

In [37]:
automaton = FiniteAutomaton()
automaton.read_input(inp)

[['Q'], ['q0', 'q1', 'q2', 'q3', 'q4'], ['E'], ['a', 'b', 'c'], ['F'], ['q4'], ['D'], ['q0', 'a', 'q1'], ['q1', 'b', 'q2'], ['q2', 'c', 'q0'], ['q1', 'b', 'q3'], ['q3', 'a', 'q4'], ['q3', 'b', 'q0']]


In [38]:
automaton.D

[['q0', 'a', 'q1'],
 ['q1', 'b', 'q2'],
 ['q2', 'c', 'q0'],
 ['q1', 'b', 'q3'],
 ['q3', 'a', 'q4'],
 ['q3', 'b', 'q0']]

In [39]:
automaton.nfa

{'q0': {'a': ['q1']},
 'q1': {'b': ['q2', 'q3']},
 'q2': {'c': ['q0']},
 'q3': {'a': ['q4'], 'b': ['q0']}}

In [40]:
automaton.to_dfa()

In [41]:
automaton.dfa

{'q0': {'a': ['q1']},
 'q1': {'b': ['q2', 'q3']},
 'q2q3': {'a': ['q4'], 'b': ['q0'], 'c': ['q0']}}

In [42]:
automaton.nfa_table

,a,b,c
q0,q1,,
q1,,q2q3,
q2,,,q0
q3,q4,q0,
q4,,,


In [31]:
automaton.dfa_table

,a,b,c
q0,q1,,
q1,,q2q3,
q2q3,q4,q0,q0
q4,,,


In [45]:
automaton.check_string("acab")

False